In [1]:
import pandas as pd

In [2]:
dataset = pd.read_csv('Social_Network_Ads.csv')

In [3]:
dataset

User ID  Gender  Age  EstimatedSalary  Purchased
0    15624510    Male   19            19000          0
1    15810944    Male   35            20000          0
2    15668575  Female   26            43000          0
3    15603246  Female   27            57000          0
4    15804002    Male   19            76000          0
..        ...     ...  ...              ...        ...
395  15691863  Female   46            41000          1
396  15706071    Male   51            23000          1
397  15654296  Female   50            20000          1
398  15755018    Male   36            33000          0
399  15594041  Female   49            36000          1

[400 rows x 5 columns]

In [4]:
dataset = pd.get_dummies(dataset,drop_first = True)

In [5]:
dataset

User ID  Age  EstimatedSalary  Purchased  Gender_Male
0    15624510   19            19000          0         True
1    15810944   35            20000          0         True
2    15668575   26            43000          0        False
3    15603246   27            57000          0        False
4    15804002   19            76000          0         True
..        ...  ...              ...        ...          ...
395  15691863   46            41000          1        False
396  15706071   51            23000          1         True
397  15654296   50            20000          1        False
398  15755018   36            33000          0         True
399  15594041   49            36000          1        False

[400 rows x 5 columns]

In [6]:
dataset.columns

Index(['User ID', 'Age', 'EstimatedSalary', 'Purchased', 'Gender_Male'], dtype='object')

In [7]:
dataset['Purchased'].value_counts()

Purchased
0    257
1    143
Name: count, dtype: int64

In [8]:
indep = dataset[['User ID', 'Age', 'EstimatedSalary', 'Gender_Male']]
dep = dataset['Purchased']

In [9]:
indep.shape

(400, 4)

In [10]:
indep

User ID  Age  EstimatedSalary  Gender_Male
0    15624510   19            19000         True
1    15810944   35            20000         True
2    15668575   26            43000        False
3    15603246   27            57000        False
4    15804002   19            76000         True
..        ...  ...              ...          ...
395  15691863   46            41000        False
396  15706071   51            23000         True
397  15654296   50            20000        False
398  15755018   36            33000         True
399  15594041   49            36000        False

[400 rows x 4 columns]

In [11]:
dep

0      0
1      0
2      0
3      0
4      0
      ..
395    1
396    1
397    1
398    0
399    1
Name: Purchased, Length: 400, dtype: int64

In [12]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(indep,dep, test_size = 0.30, random_state = 0)

In [13]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.transform(X_test)

In [14]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
param_grid = {'criterion':['gini','entropy'],
              'max_features': ['none','sqrt','log2'],
              'splitter':['best','random']}
grid=GridSearchCV(DecisionTreeClassifier(),param_grid,refit=True,verbose=3,cv=5,n_jobs=-1,scoring='f1_weighted')
grid.fit(X_train,Y_train)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


C:\Anaconda 3\Lib\site-packages\sklearn\model_selection\_validation.py:547: FitFailedWarning: 
20 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
10 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Anaconda 3\Lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Anaconda 3\Lib\site-packages\sklearn\base.py", line 1467, in wrapper
    estimator._validate_params()
  File "C:\Anaconda 3\Lib\site-packages\sklearn\base.py", line 666, in _validate_params
    validate_parameter_constraints(
  File "C:\Anaconda 3\Lib\site-packages\sklearn\utils\_param_validation.py", line 95, in valida

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_features': ['none', 'sqrt', 'log2'],
                         'splitter': ['best', 'random']},
             scoring='f1_weighted', verbose=3)

In [15]:
result=grid.cv_results_
#print(result)

Y_pred = grid.predict(X_test)
#print(y_pred)
   
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(Y_test, Y_pred)
#print(cm)

from sklearn.metrics import classification_report

clf_report = classification_report(Y_test, Y_pred)

#print(clf_report)

In [16]:
from sklearn.metrics import f1_score

f1_macro=f1_score(Y_test,Y_pred,average='weighted')

print("The f1_macro value for best parameter {}:".format(grid.best_params_),f1_macro)

The f1_macro value for best parameter {'criterion': 'gini', 'max_features': 'log2', 'splitter': 'best'}: 0.8490625


In [17]:
print("The confusion Matrix:\n",cm)

The confusion Matrix:
 [[71  8]
 [10 31]]


In [18]:
print("The report:\n",clf_report)

The report:
               precision    recall  f1-score   support

           0       0.88      0.90      0.89        79
           1       0.79      0.76      0.78        41

    accuracy                           0.85       120
   macro avg       0.84      0.83      0.83       120
weighted avg       0.85      0.85      0.85       120



In [19]:
#AUC ROC stands for “Area Under the Curve” of the “Receiver Operating Characteristic” curve. 
from sklearn.metrics import roc_auc_score

roc_auc_score(Y_test,grid.predict_proba(X_test)[:,1])

0.8274158690953999

In [20]:
table=pd.DataFrame.from_dict(result)

table

mean_fit_time  std_fit_time  mean_score_time  std_score_time  \
0        0.009287      0.005637         0.000000        0.000000   
1        0.001803      0.000751         0.000000        0.000000   
2        0.529025      0.263454         0.470135        0.228605   
3        0.004548      0.000715         0.009940        0.001995   
4        0.089961      0.056824         0.005850        0.004171   
5        0.000504      0.001009         0.013715        0.004366   
6        0.001388      0.001023         0.000000        0.000000   
7        0.002805      0.001336         0.000000        0.000000   
8        0.004661      0.003306         0.007847        0.001130   
9        0.004892      0.000851         0.009724        0.000977   
10       0.003309      0.002829         0.009239        0.001831   
11       0.003522      0.004388         0.008698        0.002807   

   param_criterion param_max_features param_splitter  \
0             gini               none           best   
1             gini               none         random   
2             gini               sqrt           best   
3             gini               sqrt         random   
4             gini               log2           best   
5             gini               log2         random   
6          entropy               none           best   
7          entropy               none         random   
8          entropy               sqrt           best   
9          entropy               sqrt         random   
10         entropy               log2           best   
11         entropy               log2         random   

                                               params  split0_test_score  \
0   {'criterion': 'gini', 'max_features': 'none', ...                NaN   
1   {'criterion': 'gini', 'max_features': 'none', ...                NaN   
2   {'criterion': 'gini', 'max_features': 'sqrt', ...           0.775048   
3   {'criterion': 'gini', 'max_features': 'sqrt', ...           0.838326   
4   {'criterion': 'gini', 'max_features': 'log2', ...           0.833024   
5   {'criterion': 'gini', 'max_features': 'log2', ...           0.737557   
6   {'criterion': 'entropy', 'max_features': 'none...                NaN   
7   {'criterion': 'entropy', 'max_features': 'none...                NaN   
8   {'criterion': 'entropy', 'max_features': 'sqrt...           0.850032   
9   {'criterion': 'entropy', 'max_features': 'sqrt...           0.799537   
10  {'criterion': 'entropy', 'max_features': 'log2...           0.833024   
11  {'criterion': 'entropy', 'max_features': 'log2...           0.690193   

    split1_test_score  split2_test_score  split3_test_score  \
0                 NaN                NaN                NaN   
1                 NaN                NaN                NaN   
2            0.802399           0.824958           0.859025   
3            0.824293           0.769053           0.839990   
4            0.840114           0.806153           0.893717   
5            0.785714           0.819142           0.875548   
6                 NaN                NaN                NaN   
7                 NaN                NaN                NaN   
8            0.769053           0.789152           0.813596   
9            0.754011           0.787755           0.875548   
10           0.782971           0.787755           0.838458   
11           0.804584           0.841398           0.839990   

    split4_test_score  mean_test_score  std_test_score  rank_test_score  
0                 NaN              NaN             NaN                9  
1                 NaN              NaN             NaN                9  
2            0.911105         0.834507        0.047172                2  
3            0.853485         0.825030        0.029476                3  
4            0.872761         0.849154        0.030770                1  
5            0.872761         0.818144        0.052584                5  
6                 NaN              NaN             NaN         